In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/data-storm-4/Data-dictionary.xlsx
/kaggle/input/data-storm-4/DataStorm4.0_KaggleProblemStatement.pdf
/kaggle/input/data-storm-4/Store-info.csv
/kaggle/input/data-storm-4/Testing-data.csv
/kaggle/input/data-storm-4/Historical-transaction-data.csv


In [2]:
trans_df = pd.read_csv("/kaggle/input/data-storm-4/Historical-transaction-data.csv",parse_dates=['transaction_date'])
trans_df["transaction_date"]=trans_df["transaction_date"].dt.date

trans_df

,item_description,transaction_date,invoice_id,customer_id,shop_id,item_price,quantity_sold
0,ORANGE BARLEY 1.5L,2021-12-11,147.0,BGXA,SHOP008,220,2
1,GINGER BEER 1.5L,2021-10-17,371.0,IA25,SHOP112,220,2
2,TONIC PET 500ML,2021-12-13,484.0,VN7V,SHOP008,160,2
3,CREAM SODA 1L,2021-12-13,484.0,VN7V,SHOP008,150,2
4,STRAWBERRY MILK 180ML,2021-10-23,1310.0,7S00,SHOP112,210,5
...,...,...,...,...,...,...,...
473969,NECTO 1.5L,2021-12-09,NaN,X21A,SHOP056,440,3
473970,LEMONADE 1.5L,2021-12-02,NaN,LV2L,SHOP018,220,2
473971,FIT O ORANGE 200ML,2021-11-19,NaN,VC4O,SHOP077,210,4
473972,CREAM SODA 1.5L,2021-12-03,NaN,MSBQ,SHOP077,440,3


In [3]:
daily_df = trans_df.drop(["item_description","invoice_id","customer_id"],axis = 1).dropna()
daily_df["income"] = daily_df["item_price"]*daily_df["quantity_sold"]
daily_df = daily_df.drop("item_price",axis = 1)
daily_df

,transaction_date,shop_id,quantity_sold,income
0,2021-12-11,SHOP008,2,440
1,2021-10-17,SHOP112,2,440
2,2021-12-13,SHOP008,2,320
3,2021-12-13,SHOP008,2,300
4,2021-10-23,SHOP112,5,1050
...,...,...,...,...
473969,2021-12-09,SHOP056,3,1320
473970,2021-12-02,SHOP018,2,440
473971,2021-11-19,SHOP077,4,840
473972,2021-12-03,SHOP077,3,1320


In [4]:
dates = daily_df.transaction_date.unique()
dates.sort()


In [5]:
store_df = pd.read_csv("/kaggle/input/data-storm-4/Store-info.csv")
store_df

,shop_id,shop_area_sq_ft,shop_profile
0,SHOP047,528,Moderate
1,SHOP009,676,High
2,SHOP083,676,Low
3,SHOP117,676,Low
4,SHOP042,676,Low
...,...,...,...
119,SHOP087,527,NaN
120,SHOP050,411,NaN
121,SHOP061,699,NaN
122,SHOP056,597,NaN


In [6]:
daily_df.loc[daily_df["shop_id"]=="SHOP001"]

,transaction_date,shop_id,quantity_sold,income
328,2021-10-16,SHOP001,1,220
332,2021-10-19,SHOP001,1,220
333,2021-10-20,SHOP001,2,280
334,2021-10-20,SHOP001,1,220
335,2021-10-24,SHOP001,2,200
...,...,...,...,...
473830,2021-10-21,SHOP001,2,300
473831,2021-10-21,SHOP001,2,300
473832,2021-10-21,SHOP001,1,150
473833,2021-10-21,SHOP001,2,300


In [7]:
for date in dates:
    day_df = daily_df.loc[daily_df["transaction_date"]==date]
    store_df[str(date)]=''
    for shop in store_df["shop_id"]:
        shop_df = day_df.loc[day_df["shop_id"]==shop]
        day_income = shop_df.income.sum()
        store_df[str(date)][store_df.index[store_df["shop_id"]==shop]] = day_income
store_df    

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


,shop_id,shop_area_sq_ft,shop_profile,2021-10-15,2021-10-16,2021-10-17,2021-10-18,2021-10-19,2021-10-20,2021-10-21,...,2021-12-06,2021-12-07,2021-12-08,2021-12-09,2021-12-10,2021-12-11,2021-12-12,2021-12-13,2021-12-14,2021-12-15
0,SHOP047,528,Moderate,18265,7475,15630,15865,16315,15375,15555,...,4175,8600,9820,15110,20955,22455,13800,21520,7600,6070
1,SHOP009,676,High,21800,36505,72570,35960,38180,32120,76185,...,27255,32210,27760,39345,68355,85750,45455,36725,65885,41095
2,SHOP083,676,Low,19250,34500,19610,58430,70610,23300,21825,...,18900,31640,32505,16655,31550,57245,30480,22500,28530,37270
3,SHOP117,676,Low,39935,37805,45125,42605,45800,35245,35235,...,31050,39460,20630,39060,43040,77040,81700,82170,73220,62090
4,SHOP042,676,Low,23390,25310,28135,17795,22240,13535,38375,...,16370,15405,22555,18285,26490,69185,30980,19525,16610,25480
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119,SHOP087,527,NaN,18840,10220,25820,27755,15835,30385,14265,...,12170,26010,11300,11150,12305,27455,20105,23250,35680,18130
120,SHOP050,411,NaN,71350,60575,28785,37520,57860,22410,27920,...,30470,157410,56830,43130,75500,70210,81755,44300,48210,85590
121,SHOP061,699,NaN,29280,80970,31820,14110,35205,22080,21145,...,33620,15730,33445,45915,71825,69675,54780,15000,30200,21340
122,SHOP056,597,NaN,52855,77155,68280,41345,93270,40095,39770,...,44225,70255,67030,43475,72025,106525,101940,46755,59905,72725


In [8]:
X = store_df.drop(["shop_profile","shop_id"],axis = 1).head(100)
y = store_df["shop_profile"].head(100)
to_be_tested = store_df.drop(["shop_profile","shop_id"],axis = 1).tail(24)

DTC

In [9]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import joblib 
from sklearn import tree

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.1)
model = DecisionTreeClassifier()
model.fit(X_train,y_train)
predictions = model.predict(X_test)

score = accuracy_score(y_test, predictions)
score

0.1

Random Forest

In [11]:
# Modelling
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, ConfusionMatrixDisplay
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from scipy.stats import randint

# Tree Visualisation
from sklearn.tree import export_graphviz
from IPython.display import Image
import graphviz

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.2)

rf = RandomForestClassifier()
rf.fit(X_train, y_train)

y_pred = rf.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.6


In [13]:
rf = RandomForestClassifier()
rf.fit(X, y)

result = rf.predict(to_be_tested)
result

array(['Low', 'Moderate', 'Low', 'High', 'Low', 'High', 'Low', 'Low',
       'Low', 'Moderate', 'Low', 'Low', 'Low', 'High', 'Moderate',
       'Moderate', 'Low', 'High', 'Moderate', 'Low', 'Low', 'Low', 'High',
       'Low'], dtype=object)

In [14]:
testing_data = pd.read_csv("/kaggle/input/data-storm-4/Testing-data.csv")
testing_data["shop_profile"] = result
testing_data

,shop_id,shop_profile
0,SHOP046,Low
1,SHOP024,Moderate
2,SHOP023,Low
3,SHOP097,High
4,SHOP044,Low
5,SHOP030,High
6,SHOP038,Low
7,SHOP029,Low
8,SHOP096,Low
9,SHOP092,Moderate


In [15]:
testing_data.to_csv("day03_result.csv")